<a href="https://colab.research.google.com/github/adbreind/accelerate-rapids/blob/master/02_RAPIDS_Blazing_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Environment Sanity Check #

Click the _Runtime_ dropdown at the top of the page, then _Change Runtime Type_ and confirm the instance type is _GPU_.

Check the output of `!nvidia-smi` to make sure you've been allocated a Tesla T4.

In [2]:
! nvidia-smi

Mon Sep 30 22:12:36 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.40       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [ ]:
! mkdir /blazingsql

In [4]:
# intall miniconda
!wget -c https://repo.continuum.io/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
!chmod +x Miniconda3-4.5.4-Linux-x86_64.sh
!bash ./Miniconda3-4.5.4-Linux-x86_64.sh -b -f -p /usr/local

# install RAPIDS packages
!conda install -q -y --prefix /usr/local -c nvidia -c rapidsai \
  -c numba -c conda-forge -c pytorch -c defaults \
  cudf=0.9 cuml=0.9 cugraph=0.9 python=3.6 cudatoolkit=10.0

# set environment vars
import sys, os, shutil
sys.path.append('/usr/local/lib/python3.6/site-packages/')
os.environ['NUMBAPRO_NVVM'] = '/usr/local/cuda/nvvm/lib64/libnvvm.so'
os.environ['NUMBAPRO_LIBDEVICE'] = '/usr/local/cuda/nvvm/libdevice/'

# copy .so files to current working dir
for fn in ['libcudf.so', 'librmm.so']:
  shutil.copy('/usr/local/lib/'+fn, os.getcwd())

--2019-09-30 22:12:42--  https://repo.continuum.io/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.200.79, 104.18.201.79, 2606:4700::6812:c84f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.200.79|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58468498 (56M) [application/x-sh]
Saving to: ‘Miniconda3-4.5.4-Linux-x86_64.sh’

Miniconda3-4.5.4-Li 100%[===================>]  55.76M  47.2MB/s    in 1.2s    

2019-09-30 22:12:44 (47.2 MB/s) - ‘Miniconda3-4.5.4-Linux-x86_64.sh’ saved [58468498/58468498]

PREFIX=/usr/local
installing: python-3.6.5-hc3d631a_2 ...
Python 3.6.5 :: Anaconda, Inc.
installing: ca-certificates-2018.03.07-0 ...
installing: conda-env-2.6.0-h36134e3_1 ...
installing: libgcc-ng-7.2.0-hdf63c60_3 ...
installing: libstdcxx-ng-7.2.0-hdf63c60_3 ...
installing: libffi-3.2.1-hd88cf55_4 ...
installing: ncurses-6.1-hf484d3e_0 ...
installing: openssl-1.0.2o-h20670df_0 ...
installing: tk

In [5]:
import nvstrings, nvcategory, cudf
import io, requests

gdf = cudf.DataFrame({'test':[1,2,3]})
print(gdf)
print(gdf.describe())

   test
0     1
1     2
2     3
       test
count   3.0
mean    2.0
std     1.0
min     1.0
25%     1.5
50%     2.0
75%     2.5
max     3.0


In [6]:
# Install BlazingSQL for CUDA 10.0
! conda install -q -y --prefix /usr/local -c conda-forge -c defaults -c nvidia -c rapidsai \
   -c blazingsql/label/cuda10.0 -c blazingsql \
   blazingsql-calcite blazingsql-orchestrator blazingsql-ral blazingsql-python

Solving environment: ...working... done

## Package Plan ##

  environment location: /usr/local

  added / updated specs: 
    - blazingsql-calcite
    - blazingsql-orchestrator
    - blazingsql-python
    - blazingsql-ral


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    six-1.12.0                 |        py37_1000          22 KB  conda-forge
    chardet-3.0.4              |        py37_1003         167 KB  conda-forge
    xorg-libxext-1.3.4         |       h516909a_0          51 KB  conda-forge
    pycparser-2.19             |           py37_1         171 KB  conda-forge
    blazingsql-calcite-0.4.3   |           py37_0        33.6 MB  blazingsql
    libtiff-4.0.10             |    h57b8799_1003         587 KB  conda-forge
    sortedcontainers-2.1.0     |             py_0          25 KB  conda-forge
    libxml2-2.9.9              |       hee79883_5         1.3 MB  conda-forge
    blaz

In [7]:
pip install flatbuffers

In [ ]:
sys.path.append('/usr/local/lib/python3.7/site-packages/')

In [1]:
from blazingsql import BlazingContext

In [2]:
bc = BlazingContext()

connection established


In [11]:
bc.create_table('foo', gdf)

Unexpected error on create_table, can only concatenate str (not "tuple") to str


In [9]:
# Query
result = bc.sql('SELECT * FROM foo ORDER BY test DESC').get()
result_gdf = result.columns

#Print GDF
print(result_gdf)

Unexpected error on _run_query_get_token


TypeError: 'NoneType' object is not iterable

In [ ]:
print('INSTALL SUCCESS!')

In [ ]:
! pwd

In [3]:
bc.create_table('beer', '/home/adam/Documents/accelerate-rapids/data/beer_small.csv')

In [4]:
result = bc.sql("SELECT * FROM beer WHERE brewery_name = 'Sunday River Brewing Co.'").get()
result_gdf = result.columns
result_gdf

,Unnamed: 0,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
0,213563,1922,Sunday River Brewing Co.,1202595455,3.5,3.0,4.0,smcolw,Dortmunder / Export Lager,4.0,3.0,Sunday River Lager,null,21923
1,213532,1922,Sunday River Brewing Co.,1187653634,4.0,3.5,3.5,BuckSpin,American Blonde Ale,3.5,3.5,Sunsplash Golden Ale,4.5,33001
2,213545,1922,Sunday River Brewing Co.,1172457299,2.5,3.5,4.0,BuckSpin,American Porter,2.5,2.5,Black Bear Porter,null,21924
3,213578,1922,Sunday River Brewing Co.,1233449031,3.5,3.5,3.5,thekevlarkid,Foreign / Export Stout,3.5,3.0,Jamaican Stout,8.0,32284


In [6]:
result = bc.sql("SELECT * FROM beer WHERE beer_style='Belgian IPA' AND review_overall > 4 ORDER BY brewery_name").get()
result_gdf = result.columns
result_gdf

,Unnamed: 0,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
0,1395906,13837,Ale Asylum,1273368858,5.0,4.0,4.0,Lukafer,Belgian IPA,4.0,4.5,Bedlam,7.5,57290
1,1395882,13837,Ale Asylum,1306508877,4.5,4.0,4.0,mikeyv35,Belgian IPA,5.0,4.5,Bedlam,7.5,57290
2,1395907,13837,Ale Asylum,1273281879,4.5,4.0,4.0,alleykatking,Belgian IPA,4.5,4.0,Bedlam,7.5,57290
3,1395883,13837,Ale Asylum,1305942919,4.5,4.5,4.0,brewcrew76,Belgian IPA,4.0,4.0,Bedlam,7.5,57290
4,1220885,4,Allagash Brewing Company,1256558384,4.5,4.5,4.0,Wasatch,Belgian IPA,4.5,4.5,Allagash Hugh Malone,8.5,36527
5,1220833,4,Allagash Brewing Company,1177637832,4.5,4.0,4.5,ppoitras,Belgian IPA,4.0,4.0,Allagash Hugh Malone,8.5,36527
6,1220910,4,Allagash Brewing Company,1257861782,4.5,4.0,4.5,GrittyMug345,Belgian IPA,3.5,4.5,Allagash Hugh Malone,8.5,36527
7,1220782,4,Allagash Brewing Company,1196821969,4.5,4.0,4.0,shroompod,Belgian IPA,4.0,4.5,Allagash Hugh Malone,8.5,36527
8,1220708,4,Allagash Brewing Company,1242100571,5.0,4.5,4.5,Chodite,Belgian IPA,5.0,5.0,Allagash Hugh Malone,8.5,36527
9,1220858,4,Allagash Brewing Company,1295837565,4.5,4.5,4.0,americanpsycho,Belgian IPA,4.0,4.5,Allagash Hugh Malone,8.5,36527
